In [1]:
import pandas as pd

In [2]:
df = pd.read_spss("tastdb-exp-2020.sav")

In [3]:
# HELPER FUNCTIONS
#   Call these functions to help understand the data

def display_columns():
    ''' Displays all column names '''
    
    cols = list(df.columns)
    for col in cols:
        print(col)

def display_top_importers():
    ''' Display the top importers sorted by total number of slaves imported'''
    
    voyage_by_port = voyage_dict.sort_values(by="SLAMIMP", ascending=False).groupby("MJSELIMP", observed=True).sum()
    voyage_by_port = voyage_by_port.apply(lambda x : x.sort_values(ascending=False))
    print(str(len(voyage_by_port)) + " destination ports")
    voyage_by_port.head(len(voyage_by_port))

def display_num_nans():
    ''' Display the number of Nans that appear in each column '''
    
    print("Attribute\t\tNaNs")
    print("-"*40)
    cols = list(df.columns)
    for col in cols:
        if df[col].isna().sum() < 15000:
            print(str(col) + ":\t\t" + str(df[col].isna().sum()))
            
def ports_map():
    fig2 = px.scatter_geo(voyage_dict, size=voyage_dict["SLAMIMP"])

    fig2.update_traces(
        lat=[arrive_coords[port][0] for port in arrive_coords],
        lon=[arrive_coords[port][1] for port in arrive_coords],
        marker = dict(
            color = "green",
            size=voyage_dict["SLAMIMP"]
        )
    )

    fig2.update_geos(
        center = dict(lat=15, lon=-32),
        lataxis_range=[-50,50], lonaxis_range=[-110, 60]
        )


    fig1 = px.scatter_geo(voyage_dict, size=voyage_dict["SLAMIMP"])

    fig1.update_traces(
        lat=[depart_coords[port][0] for port in depart_coords],
        lon=[depart_coords[port][1] for port in depart_coords],
        marker = dict(
            color = "red",
            size=voyage_dict["SLAMIMP"]
        )
    )

    fig1.update_geos(
        center = dict(lat=15, lon=-32),
        lataxis_range=[-50,50], lonaxis_range=[-110, 60]
        )
    
    return fig1, fig2

In [4]:
voyage_dict = df[["MJBYPTIMP", "MJSELIMP", "SLAMIMP"]].dropna()

voyage_groups = voyage_dict.groupby("MJSELIMP", observed=True)
voyage_groups_dict = {group[0].lstrip():group[1] for group in voyage_groups}

voyage_by_port = voyage_dict.groupby("MJSELIMP", observed=True)[["SLAMIMP"]].sum()
print(len(voyage_by_port))

72


In [5]:
import plotly.graph_objects as go
import plotly.express as px
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="MyApp")
import json


def save_coords():
    
    depart_coords = {city.lstrip():get_coordinates(city) for city in voyage_dict["MJBYPTIMP"].unique()}
    with open("depart_coords.txt", 'w') as file:
        file.write(json.dumps(depart_coords))
    print("Step 1 complete!")
    
    arrive_coords = {city.lstrip():get_coordinates(city) for city in voyage_dict["MJSELIMP"].unique()}
    with open("arrive_coords.txt", 'w') as file:
        file.write(json.dumps(arrive_coords))
    print("Step 2 complete!")
    
    
    
def read_coords_from_file():
    
    with open("depart_coords.txt", 'r') as file:
        depart_coords = json.loads(file.read())
        
    with open("arrive_coords.txt", 'r') as file:
        arrive_coords = json.loads(file.read())
        
    return depart_coords, arrive_coords
    
    

def get_coordinates(city):
    try:
        location = geolocator.geocode(city)
        loc = (location.latitude, location.longitude)
        #print(city + ":\t" + str(loc))
        return loc
    except:
        return "NOT FOUND"
    
depart_coords, arrive_coords = read_coords_from_file()

In [61]:
groups = [group for group in voyage_groups]
groups = [groups[i][0].lstrip() for i in range(len(groups))]
miscellaneous = ['Codes without labels', 'Bight of Benin', 'British Americas', 'Gold Coast', 'Netherlands', 'Other Brazil', 'Other Africa', 'Peru', 'England', 'France']
for misc in miscellaneous:
    groups.remove(misc)

time_series = df[["YEARAM", "MJBYPTIMP", "MJSELIMP", "SLAMIMP"]].dropna()

def display_graph(group_name):
    country_series = time_series[time_series["MJSELIMP"] == " " + group_name]
    x = country_series["YEARAM"]
    y = country_series["SLAMIMP"]

    f = px.bar(
        x=x, y=y,
        title="Transportation of Enslaved Persons",
        labels={"x": "Year", "y": "Number of Persons"},
        template="plotly_dark",
        opacity=0.85,
        color_discrete_sequence=["#b80e02"],
    )

    f.update_layout(
        title_font=dict(size=22, color="#b80e02"),
        xaxis=dict(
            showticklabels=True,
            tickfont=dict(size=14, color="white"),
            title_font=dict(size=16, color="white"),
        ),
        yaxis=dict(
            tickfont=dict(size=14, color="white"),
            title_font=dict(size=16, color="white"),
        ),
        plot_bgcolor="#121212",
        paper_bgcolor="#121212",
    )

    return f

def plot_voyage_group(group_name):
    fig = px.scatter_geo(
        voyage_dict, 
        size=voyage_dict["SLAMIMP"], 
        template="plotly_dark",
        color_discrete_sequence=["#b80e02"],
    )

    fig.update_layout(
        title={
            'text': "Dispersion of Enslaved Persons",
            'font': dict(size=22, color="#b80e02"),
        },
        geo=dict(
            showcoastlines=True, coastlinecolor="gray",
            showland=True, landcolor="#1E1E1E",
            showocean=True, oceancolor="#2E2E2E", 
            projection_type="natural earth",
        )
    )

    group = voyage_groups_dict[group_name]
    for voyage in group["MJBYPTIMP"]:
        try:
            fig.add_trace(
                go.Scattergeo(
                    lon=[depart_coords[voyage][1], arrive_coords[" "+group_name][1]],
                    lat=[depart_coords[voyage][0], arrive_coords[" "+group_name][0]],
                    mode="lines",
                    line=dict(width=1.5, color="#b80e02"),  # Gold lines
                    opacity=0.7,
                    hoverinfo="text",
                )
            )
        except:
            continue

    fig.update_traces(showlegend=False)
    return fig


def find_top_imports(group_name):
    imports = list(voyage_groups_dict[group_name]["MJBYPTIMP"].unique())
    imports_dict = {imp:0 for imp in imports}
   
    group = voyage_groups_dict[group_name]
    ports = group.groupby("MJBYPTIMP", observed=True)[["SLAMIMP"]].sum().sort_values(by="SLAMIMP", ascending=False)
    #ports = group.groupby("MJBYPTIMP").sum().sort_values(by="SLAMIMP", ascending=False)
    return ports.head(10)


import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

app.layout = html.Div(style={'background-color': '#121212', 'color': '#b80e02', 'font-family': 'Georgia, serif', 'padding': '20px'}, children=[
   
    html.H1("Trans-Atlantic Slave Trade", 
            style={'text-align': 'center', 'font-family': 'Georgia, serif', 
                   'font-size': '3em', 'color': '#b80e02', 'margin-bottom': '10px'}),
    
    html.H3("The figures below depict the number of enslaved persons transported, where they came from, and where they were taken between 1514 and 1866.",
            style={'text-align': 'center', 'font-family': 'Arial, sans-serif', 
                   'font-size': '1.2em', 'color': '#D0D0D0', 'max-width': '800px', 'margin': 'auto'}),
    
    html.H4("Choose a region", 
            style={'text-align': 'center', 'font-family': 'Georgia, serif', 
                   'font-size': '1.5em', 'color': '#FFFFFF', 'margin-top': '20px'}),
    
    html.Div(
        dcc.Dropdown(
            groups, 
            'Bahia', 
            id='dropdown-selection', 
            searchable=False, 
            clearable=False,
            style={
                #'display': 'inline-block',  # Prevents full-width stretching
                'width': '100%',  # Make it fully use the div width
                'height': '40px',
                'font-family': 'Arial, sans-serif',
                'font-size': '1.2em',
                'color': '#b80e02',
                'background-color': 'black',
                'border': '0.5px solid #b80e02',
                'border-radius': '5px',
                'text-align': 'center',
                'padding': '0px'
            }
        ), 
        style={
            'display': 'flex', 
            'justify-content': 'center', 
            'width': '30%',  # Constrains the background width
            'margin': 'auto'  # Centers it
        }
    ),
    
    html.Div([
        dcc.Graph(id='graph2', style={'flex': 1, 'padding': '10px'}),
        dcc.Graph(id='graph3', style={'flex': 1, 'padding': '10px'})
    ], style={'display': 'flex', 'justify-content': 'center', 'width': '100%', 'gap': '10px'}),
    
    dcc.Graph(id='graph1', style={'padding-top': '20px'})
])

@app.callback(
    Output('graph2', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    fig = plot_voyage_group(value)
    return fig

@app.callback(
    Output('graph1', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    return display_graph(value)

@app.callback(
    Output('graph3', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    df = find_top_imports(value)
    f = px.bar(
        y=df.index, x=df["SLAMIMP"],
        template="plotly_dark",
        labels={"x": "", "y": ""},
        opacity=0.85,
        orientation="h", 
        color_discrete_sequence=["#b80e02"],
    )

    f.update_layout(
        yaxis=dict(
            autorange="reversed",
            tickfont=dict(size=14, color="white"),
        ),
        xaxis=dict(
            tickfont=dict(size=14, color="white"),
        ),
        title={
            "text": "Top Imports of Enslaved Persons",
            "x": 0.5,
            "y": 0.95,
            "font": dict(size=22, color="#b80e02"),
        },
        plot_bgcolor="#121212",
        paper_bgcolor="#121212",
    )

    return f
   

if __name__ == '__main__':
    app.run_server(host='0.0.0.0', port=8080, debug=False, use_reloader=False)